In [2]:
# Data
import numpy as np
import pandas as pd
excel_file = "data.xlsx"
preference_matrix = pd.read_excel(excel_file)


In [3]:
# Scheme 1- Voting for one (plurality voting) -- [1,0,0,0,...,0] {Sai}
# VOTING PREFERENCE-PLURALITY VOTING
#HELPER FUNCTIONS
def distance(colom,result,desired):
    temp = colom[colom == desired]
    index = temp.index
    winposition_in_true_pref = int(index[0][0])
    return winposition_in_true_pref - 1

#HAPPINESS CALCULATOR FOR PLURALITY
def HappinessCalculator(preference_matrix,winner):
    weight = preference_matrix.shape[0] #ONLY FOR PLURALITY
    d = 0
    global_happiness = 0
    for (indexlabel,column_series) in preference_matrix.iteritems():
        for item in column_series.iteritems():
            if(item[1] == winner):
                global_happiness+=1
                break
            else:
                dist = distance(column_series,item[1],winner)
                weight_voter = weight -dist
                d= 1/(1+dist*weight_voter)
                global_happiness+= d
                break
            weight-=1
        weight = preference_matrix.shape[0]
    print('Happiness of the voters is : ' + str(global_happiness))
    
def UpdateWinnerCheck(matrix,considered,voter,index_of_voter,winner):
    first_pref = matrix.at['1st preference', voter]
    matrix.at[index_of_voter,voter] = first_pref
    matrix.at['1st preference', voter] = considered
    w,x = PluralityWinner(matrix)
    return w,matrix
    
def CompareWinners(preference_matrix, winner, considered_winner):
    count = 0
    s = 0
    for voter in preference_matrix.iteritems():
        voter_data = voter[1]
        temp = voter_data[voter_data == winner]
        index = temp.index
        if(index[0] != "10th preference"):
            win_position = int(index[0][0])
        else:
            win_position = 10
        temp2 = voter_data[voter_data == considered_winner]
        index2 = temp2.index
        if(index2.size == 0):
            print("Something wrong...voter preferences are not unique")
        if(index2[0] != "10th preference"):
            consider_position = int(index2[0][0])
        else:
            consider_position = 10
        if(consider_position < win_position):
            matrix_copy = preference_matrix.copy()
            updated_winner,updated_matrix=UpdateWinnerCheck(matrix_copy,considered_winner,voter[0],index2,winner)
            if(updated_winner != winner):
                print('Tactical voting possible because ' + considered_winner + ' is preferred over ' + winner +' by ' + voter[0])
                count+=1
                s+=1
                print('Updated winner after tactical voting is '+str(updated_winner))
                print('Updated preference list for '+voter[0])
                print(updated_matrix[voter[0]].T)
                HappinessCalculator(updated_matrix,updated_winner)
    if(count == 0):
            print('Tactical voting not possible for '+considered_winner+' due to Lexographic rule or no voter prefers him over the winner')
    return s

#DETERMINES WINNER FOR PLURALITY
def PluralityWinner(preference_matrix):
    winner_row = preference_matrix.T.sort_values('1st preference')
    unsorted_winrow=winner_row.sort_values('1st preference')['1st preference'].value_counts()
    sorted_winrow = unsorted_winrow.sort_index()
    winner = sorted_winrow.idxmax()
    return winner, unsorted_winrow

def TacticalVoting(preference_matrix, unsorted_winrow, winner):
    mods = 0
    first_row = preference_matrix.loc['1st preference']
    if(unsorted_winrow[0]>unsorted_winrow[1]+1): #WITHOUT VOTER COLLUSION
        print("Tactical Voting Not Possible In this Case without VOTER COLLUSION")
        return
    print("Tactical voting may be possible if somebody prefers other than the winner and wins with lexographic rule")
    winner_count = unsorted_winrow[0]
    possible_winners = unsorted_winrow[(unsorted_winrow == winner_count - 1) | (unsorted_winrow == winner_count)]
    for item in possible_winners.iteritems():
        if(item[0] == winner):
            #print("No tactical voting for winner as an alternative")
            pass
        else:
            considered_winner = item[0]
            print('Checking if ' + considered_winner + ' is preferred over the winner')
            mods+=CompareWinners(preference_matrix,winner,considered_winner)
            pass
        pass
    return mods



# VOTING PREFERENCE-PLURALITY VOTING
winner, unsorted_winrow = PluralityWinner(preference_matrix)
print('According to plurality voting the winner is ' + winner )
HappinessCalculator(preference_matrix,winner)
mods = TacticalVoting(preference_matrix,unsorted_winrow,winner)
risk = mods/preference_matrix.shape[1]
print('Risk of tactical voting according to plurality voting scheme is '+str(risk))


According to plurality voting the winner is A
Happiness of the voters is : 2.4175647881530233
Tactical voting may be possible if somebody prefers other than the winner and wins with lexographic rule
Checking if F is preferred over the winner
Tactical voting possible because F is preferred over A by 2nd voter
Updated winner after tactical voting is F
Updated preference list for 2nd voter
1st preference     F
2nd preference     G
3rd preference     B
4th preference     H
5th preference     C
6th preference     E
7th preference     D
8th preference     A
9th preference     I
10th preference    J
Name: 2nd voter, dtype: object
Happiness of the voters is : 3.282377622377622
Tactical voting possible because F is preferred over A by 10th voter
Updated winner after tactical voting is F
Updated preference list for 10th voter
1st preference     F
2nd preference     H
3rd preference     D
4th preference     I
5th preference     C
6th preference     B
7th preference     G
8th preference     A
9th 

In [ ]:
# Scheme 2- Voting for two -- [1,1,0,0,...,0] {Tom}

In [15]:
# Scheme 3- Anti-plurality voting (veto) -- [1,1,1,1,...,0] {Zhaolin}

'B'

In [7]:
# Scheme 4- Borda voting -- [m-1,m-2,m-2,m-3,...,0] {Chinmay}
original_pref_matrix = preference_matrix
n_candidates = original_pref_matrix.shape[0]
n_voters = original_pref_matrix.shape[1]

def getOutcome(pref_matrix):
    candidate_dict = {'A':0, 'B':0, 'C':0, 'D':0, 'E':0, 'F':0, 'G':0, 'H':0, 'I':0, 'J':0}

    for v in range(n_voters):
        for c in range(n_candidates):
            candidate_dict[pref_matrix.T.iloc[v,c]] += n_candidates-1-c
    outcome_list = [(k,v) for k,v in candidate_dict.items()]
    outcome_list.sort(key=lambda val:val[1], reverse=True)

    return np.array(outcome_list)

# Happiness calculation
def getHappiness(outcome_list):
    W = list(range(1, original_pref_matrix.shape[1]+1)) # weights
    W.reverse()

    total_happiness = 0
    pref_matrix = original_pref_matrix.values
    for v in range(n_voters):
        voter_pref_list = pref_matrix[:,v]
        d_i = 0
        for c in range(n_candidates):
            k = n_candidates - np.argwhere(outcome_list == voter_pref_list[c])[0][0] # position of the alt in outcome (indexed form bottom)
            j = n_candidates - c # position of the alt in the voter's pref (indexed form bottom)
            d_ij = k - j
            d_i += W[c]*d_ij
        voter_happiness = 1/(1+abs(d_i))
        total_happiness += voter_happiness
    return total_happiness

true_outcome = getOutcome(original_pref_matrix)
true_happiness = getHappiness(true_outcome_list)

print("True (non-strategic) outcome: \n", true_outcome)
print("True (non-strategic) happiness: ", true_happiness)

True (non-strategic) outcome: 
 [['A' '57']
 ['B' '56']
 ['G' '55']
 ['F' '51']
 ['C' '50']
 ['D' '46']
 ['H' '45']
 ['E' '36']
 ['I' '36']
 ['J' '18']]
True (non-strategic) happiness:  0.233491391289
